In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

tf.set_random_seed(777)

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10 # Window size. any arbitrary number is given.
learning_rate = 0.1

In [3]:
dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i: i+sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)
    
batch_size = len(dataX)

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

# one-hot encoding
X_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
print(X_one_hot)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [4]:
# Make a lstm cell with hidden_size (each unit output vector size)
def lstm_cell():
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

multi_cells = rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

In [5]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)

# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

0 0 mus,ssssss 3.21861
0 1 nnnnssssss 3.21861
0 2 m,wsssssss 3.21861
0 3 ,wwsssssss 3.21861
0 4 hrrrssssss 3.21861
0 5 'rssssssss 3.21861
0 6 msssssssss 3.21861
0 7 asssssssss 3.21861
0 8 rsssyaaaas 3.21861
0 9  yyyywaass 3.21861
0 10 ysyasassss 3.21861
0 11 mmsssrrrrr 3.21861
0 12 yawarrrrrr 3.21861
0 13 hrrrrrrrrr 3.21861
0 14 mmrrrmmmmm 3.21861
0 15 ccrr'mmmmn 3.21861
0 16 ''m'mmmmnm 3.21861
0 17 mmmmmmmmmm 3.21861
0 18 stmnnnnnnm 3.21861
0 19 ffmnnnnnmm 3.21861
0 20 m,rnnnnnrm 3.21861
0 21 rr,nnnnnnm 3.21861
0 22 mmkmmmmmmm 3.21861
0 23 pkk  mmmmm 3.21861
0 24 bamrrrrmms 3.21861
0 25 m,rrrrrrrs 3.21861
0 26 ,rrrrrrsss 3.21861
0 27 rrrrwwwwww 3.21861
0 28 mmwwwwswww 3.21861
0 29 ffwwwwwwww 3.21861
0 30 hihssswwws 3.21861
0 31  essswwyyy 3.21861
0 32 sssnnnnnnn 3.21861
0 33 ysmmyyyy,, 3.21861
0 34 mmmfmmm,,, 3.21861
0 35 fffffy,,,, 3.21861
0 36 ffy,,,,,,, 3.21861
0 37 'rrrr,,,nr 3.21861
0 38 hsrrr,rrrr 3.21861
0 39 mrr,,,rrrr 3.21861
0 40 'r,,,,rrrr 3.21861
0 41 ,,,,rrrrrr 3.21861
0 

2 141            3.2672
2 142            3.2672
2 143            3.2672
2 144            3.2672
2 145            3.2672
2 146            3.2672
2 147            3.2672
2 148            3.2672
2 149            3.2672
2 150            3.2672
2 151            3.2672
2 152            3.2672
2 153            3.2672
2 154            3.2672
2 155            3.2672
2 156            3.2672
2 157            3.2672
2 158            3.2672
2 159            3.2672
2 160            3.2672
2 161            3.2672
2 162            3.2672
2 163            3.2672
2 164            3.2672
2 165            3.2672
2 166            3.2672
2 167            3.2672
2 168            3.2672
2 169            3.2672
3 0            2.98545
3 1            2.98545
3 2            2.98545
3 3            2.98545
3 4            2.98545
3 5            2.98545
3 6            2.98545
3 7            2.98545
3 8            2.98545
3 9            2.98545
3 10            2.98545
3 11            2.98545
3 12            2.98545
3 

5 8 t          2.90719
5 9 t          2.90719
5 10 t          2.90719
5 11 t          2.90719
5 12 t          2.90719
5 13 t          2.90719
5 14 t          2.90719
5 15 t          2.90719
5 16 t          2.90719
5 17 t          2.90719
5 18 t          2.90719
5 19 t          2.90719
5 20 t          2.90719
5 21 t          2.90719
5 22 t          2.90719
5 23 t          2.90719
5 24 t          2.90719
5 25 t          2.90719
5 26 t          2.90719
5 27 t          2.90719
5 28 t          2.90719
5 29 t          2.90719
5 30 t          2.90719
5 31 t          2.90719
5 32 t          2.90719
5 33 t          2.90719
5 34 t          2.90719
5 35 t          2.90719
5 36 t          2.90719
5 37 t          2.90719
5 38 t          2.90719
5 39 t          2.90719
5 40 t          2.90719
5 41 t          2.90719
5 42 t          2.90719
5 43 t          2.90719
5 44 t          2.90719
5 45 t          2.90719
5 46 t          2.90719
5 47 t          2.90719
5 48 t          2.90719
5 49 t          2.

8 11 t          2.86426
8 12            2.86426
8 13 t          2.86426
8 14 t          2.86426
8 15 t          2.86426
8 16 t          2.86426
8 17            2.86426
8 18            2.86426
8 19            2.86426
8 20 t          2.86426
8 21            2.86426
8 22 t          2.86426
8 23            2.86426
8 24            2.86426
8 25            2.86426
8 26            2.86426
8 27 t          2.86426
8 28 t          2.86426
8 29            2.86426
8 30 t          2.86426
8 31 t          2.86426
8 32 t          2.86426
8 33            2.86426
8 34 t          2.86426
8 35            2.86426
8 36            2.86426
8 37 t          2.86426
8 38            2.86426
8 39 t          2.86426
8 40 t          2.86426
8 41            2.86426
8 42 t          2.86426
8 43            2.86426
8 44            2.86426
8 45 t          2.86426
8 46            2.86426
8 47            2.86426
8 48            2.86426
8 49 t          2.86426
8 50            2.86426
8 51 t          2.86426
8 52            

11 0            2.83431
11 1            2.83431
11 2            2.83431
11 3            2.83431
11 4            2.83431
11 5            2.83431
11 6            2.83431
11 7            2.83431
11 8            2.83431
11 9            2.83431
11 10            2.83431
11 11            2.83431
11 12            2.83431
11 13            2.83431
11 14            2.83431
11 15            2.83431
11 16            2.83431
11 17            2.83431
11 18            2.83431
11 19            2.83431
11 20            2.83431
11 21            2.83431
11 22            2.83431
11 23            2.83431
11 24            2.83431
11 25            2.83431
11 26            2.83431
11 27            2.83431
11 28            2.83431
11 29            2.83431
11 30            2.83431
11 31            2.83431
11 32            2.83431
11 33            2.83431
11 34            2.83431
11 35            2.83431
11 36            2.83431
11 37            2.83431
11 38            2.83431
11 39            2.83431
11 40     

...

487 73  and don't 0.229424
487 74 tnd won't  0.229424
487 75 nd won't a 0.229424
487 76 d won't as 0.229424
487 77  aon't ass 0.229424
487 78 ton't assi 0.229424
487 79  n't dssig 0.229424
487 80  't dssign 0.229424
487 81 dt dssign  0.229424
487 82 t dssign t 0.229424
487 83  dssign th 0.229424
487 84 tnsign the 0.229424
487 85 nkign them 0.229424
487 86   gn them  0.229424
487 87  tn them t 0.229424
487 88 ln them ta 0.229424
487 89   them tas 0.229424
487 90 dthem task 0.229424
487 91 toem tasks 0.229424
487 92  em tasks  0.229424
487 93 em tasks a 0.229424
487 94   tosks an 0.229424
487 95  tasks and 0.229424
487 96 tosks and  0.229424
487 97  sks and w 0.229424
487 98 nks and wo 0.229424
487 99  s and wor 0.229424
487 100 s and work 0.229424
487 101  and work, 0.229424
487 102 tnd work,  0.229424
487 103 nd work, b 0.229424
487 104 d work, bu 0.229424
487 105  aork, but 0.229424
487 106 took, but  0.229424
487 107 ook, but r 0.229424
487 108   , but ra 0.229424
487 109  , but rat 

490 36  m up peop 0.229411
490 37   up peopl 0.229411
490 38  tp people 0.229411
490 39 tp people  0.229411
490 40   people t 0.229411
490 41 epeople to 0.229411
490 42 teople tog 0.229411
490 43 eople toge 0.229411
490 44  ple toget 0.229411
490 45  le togeth 0.229411
490 46 ee togethe 0.229411
490 47 estogether 0.229411
490 48  eogether  0.229411
490 49 to ether t 0.229411
490 50 h ether to 0.229411
490 51  ether to  0.229411
490 52  ther to c 0.229411
490 53  her to co 0.229411
490 54 hem te col 0.229411
490 55 em te coll 0.229411
490 56   te colle 0.229411
490 57  te collec 0.229411
490 58 to collect 0.229411
490 59 h collect  0.229411
490 60  collect w 0.229411
490 61 tollect wo 0.229411
490 62 hllect woo 0.229411
490 63  lect wood 0.229411
490 64 eect wood  0.229411
490 65 est wood a 0.229411
490 66  t wood an 0.229411
490 67 h wood and 0.229411
490 68 hdood and  0.229411
490 69 tood and d 0.229411
490 70 ood and do 0.229411
490 71  d and don 0.229411
490 72   and don' 0.229411
4

493 42 teople tog 0.229375
493 43 ,ople toge 0.229375
493 44  ple toget 0.229375
493 45 nle togeth 0.229375
493 46 ,e togethe 0.229375
493 47 estogether 0.229375
493 48  together  0.229375
493 49 to ether t 0.229375
493 50   ether to 0.229375
493 51 nether to  0.229375
493 52  ther to c 0.229375
493 53  her to co 0.229375
493 54  em te col 0.229375
493 55 em te coll 0.229375
493 56   te colle 0.229375
493 57  te collec 0.229375
493 58 to collect 0.229375
493 59   collect  0.229375
493 60 ncollect w 0.229375
493 61 tollect wo 0.229375
493 62 ollect woo 0.229375
493 63 nlect wood 0.229375
493 64 eect wood  0.229375
493 65 est wood a 0.229375
493 66  t wood an 0.229375
493 67 o wood and 0.229375
493 68  dood and  0.229375
493 69 tood and d 0.229375
493 70 ood and do 0.229375
493 71 nd and don 0.229375
493 72 n and don' 0.229375
493 73  and don't 0.229375
493 74 tnd won't  0.229375
493 75 nd won't a 0.229375
493 76 d won't as 0.229375
493 77  aon't ass 0.229375
493 78 ton't dssi 0.229375
4

496 52  ther to c 0.229355
496 53  her to co 0.229355
496 54  em te col 0.229355
496 55 em te coll 0.229355
496 56   te colle 0.229355
496 57  to collec 0.229355
496 58 to lollect 0.229355
496 59   lollect  0.229355
496 60 nlollect w 0.229355
496 61 tollect wo 0.229355
496 62 hllect woo 0.229355
496 63 nlect wood 0.229355
496 64 eect wood  0.229355
496 65 est wood a 0.229355
496 66  t wood an 0.229355
496 67 h wood and 0.229355
496 68  dood and  0.229355
496 69 tood and d 0.229355
496 70 ood and do 0.229355
496 71 nd and don 0.229355
496 72 n and don' 0.229355
496 73  and don't 0.229355
496 74 tnd don't  0.229355
496 75 nd don't a 0.229355
496 76 d don't as 0.229355
496 77  aon't ass 0.229355
496 78 ton't assi 0.229355
496 79  n't assig 0.229355
496 80 n't assign 0.229355
496 81 dt dssign  0.229355
496 82 t assign t 0.229355
496 83  dssign th 0.229355
496 84 tnsign the 0.229355
496 85 nkign them 0.229355
496 86   gn them  0.229355
496 87  gn them t 0.229355
496 88 ln them ta 0.229355
4

498 105  aork, but 0.229346
498 106 tork, but  0.229346
498 107 ork, but r 0.229346
498 108  k, but ra 0.229346
498 109  , but rat 0.229346
498 110 s but rath 0.229346
498 111  but rathe 0.229346
498 112 tut rather 0.229346
498 113 ut rather  0.229346
498 114 t rather t 0.229346
498 115 haather te 0.229346
498 116 tather tea 0.229346
498 117  ther teac 0.229346
498 118 nher teach 0.229346
498 119 her toach  0.229346
498 120 er toach t 0.229346
498 121   toach th 0.229346
498 122  toach the 0.229346
498 123 toach them 0.229346
498 124 hach them  0.229346
498 125  ch them t 0.229346
498 126 nh them to 0.229346
498 127 t them to  0.229346
498 128 ethem to l 0.229346
498 129 toem ta lo 0.229346
498 130 her to lon 0.229346
498 131 er ta long 0.229346
498 132   ta long  0.229346
498 133  ta long f 0.229346
498 134 to bong fo 0.229346
498 135 h bong for 0.229346
498 136  cong for  0.229346
498 137 tong for t 0.229346
498 138 eng for th 0.229346
498 139  ' for the 0.229346
498 140 d for the  0